# Сервисные функции 

не нужные для работы моделей, но нужные для обслуживания

In [19]:
from cmasf import pandas_sql as pds

%run common.ipynb # загрузка общих функций и констант, все, что оттуда, должно иметь префикс common.

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strYearDBPath)) # connection к рабочей базе данных
conWorkEx = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExYearDBPath)) # connection к к рабочей базе данных экзогенных переменных
conWorkExH=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExParamDBPath)) # connection к рабочей базе данных экзогенных параметров
conWorkSvod=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strSvodDBPath)) # connection к рабочей базе данных СВОД


<div class="alert alert-block alert-danger">
    <b>ПОКА НЕ ОТЛАЖЕНО!!! КОПИЮ БАЗЫ ДАННЫХ ПЕРЕД ОБНОВЛЕНИЕМ ДЕЛАТЬ ОБЯЗАТЕЛЬНО!!!</b>
</div>

In [157]:
import pandas as pd

class service:
    
    def make_headers4db(pdfHeaders, con=None):
        pdfH=pdfHeaders.copy()
        pdfh=pd.read_sql('select * from headers', con=con, index_col='code')
        max_index=pdfh.index.max()

        dct={v['code2']:int(v['code']) for _, v in pdfh.reset_index().iterrows()}
        pdfH['code']=pdfH['code2'].map(dct)
        len_na=pdfH[pdfH['code'].isna()].shape[0]
        pdfH.loc[pdfH['code'].isna(), 'code']=range(max_index+1, max_index+1+len_na)
        pdfH['code']=pdfH['code'].round().astype(int)
        pdfH.set_index('code', inplace=True)
        return pdfH
    
    def make_datas4db(pdfData, con=None):
        pdf=pdfData.stack().reset_index().rename(columns={'level_0':'date', 'level_1':'code2', 0:'value'})
        pdfH=service.make_headers4db(pdfH1, con=con)

        pdf['code']=pdf['code2'].map({v['code2']:v['code'] for _, v in pdfH.reset_index().iterrows()})
        pdf.set_index(['code', 'date'], inplace=True)
        return pdf[['value']]
    
    def update_db(pdfData, pdfHeader=None, con=None, data_table='datas', header_table='headers'):
        
        cols=pdfData.columns.to_list()
#         ltmp=['p_sq1room_x', 'p_sq3room_x']
        strChecHeaders='select * from {headers} where code2 in {list_fields}'.format(headers=header_table, list_fields=tuple(cols))
        ph=pd.read_sql(strChecHeaders, con=con)
        
        diff=set(cols) - set(ph.code2.unique())
        if diff != set():
            strErr='В базе данных "{db_name}" нет следующих рядов: {lst}'.format(lst=diff, db_name=con.url.database)
            if pdfHeader is None:
                raise NameError('{}, необходим фрейм с заголовками рядов'.format(strErr))
            elif set(set(cols) - set(pdfHeader.code2.unique())) != set():
                raise NameError('{}, фрейм с заголовками рядов не содержит данных для {}'.format(strErr, 
                                                                                                 set(cols) - set(pdfHeader.code2.unique())))
        pdfH=pds.DataFrameDATA(service.make_headers4db(pdfHeader, con=con))
        pdfH['update_time']=common.update_dt_s(datetime_value=dt.datetime.now())
        pdfH.to_sql(header_table, con=con, if_exists='upsert')
        pdfs=pds.DataFrameDATA(service.make_datas4db(pdfData, con=con))
        pdfs.to_sql(data_table, con=con, if_exists='upsert')
        print('write to {}'.format(con.url.database), 'done')
        print(header_table, '\n', pdfH)
        print(data_table, '\n', pdfs)
        


## Ниже - тесты функций. Чтобы работало, нужно убрать магию

In [158]:
# %%script false --no-raise-error


pdf=pds.DataFrameDATA({'MEPop':[10.1775090888518,10.2349395704228,10.3011821606418,10.3712411806194,10.4456277754351,10.5221830043789,
10.5981795394509,10.674408483961,10.7487557680358,10.8222068116406,10.8926895654648],
'DispPop':[0.76449173,0.76449173,0.76449173,0.76449173,0.76449173,0.76449173,0.76449173,0.76449173,0.76449173,
0.76449173,0.76449173]}, index=range(2020, 2031))


pdfH1=pd.DataFrame([{'code2': 'MEPop', 'name': 'Параметр а логнормального распределения'} , 
                   {'code2': 'DispPop', 'name': 'Параметр sigma логнормального распределения'}, 
                  {'code2': 'p_sq1room_x', 'name': 'однокомнатная'}])
           

# print(make_headers(pdfH1, con=conWorkExH))

# print(pdfH)

# print(service.make_datas4db(pdf, con=conWorkExH))

service.update_db(pdf, con=conWorkExH, pdfHeader=pdfH)



write to ../DB/exog_param.sqlite3 done
headers 
             code2                                         name  \
code                                                             
111         MEPop      Параметр а логнормального распределения   
112       DispPop  Параметр sigma логнормального распределения   
27    p_sq1room_x                                однокомнатная   

                     update_time  
code                              
111   2021-01-27 17:50:41 607317  
112   2021-01-27 17:50:41 607317  
27    2021-01-27 17:50:41 607317  
datas 
                value
code date           
111  2020  10.177509
112  2020   0.764492
111  2021  10.234940
112  2021   0.764492
111  2022  10.301182
112  2022   0.764492
111  2023  10.371241
112  2023   0.764492
111  2024  10.445628
112  2024   0.764492
111  2025  10.522183
112  2025   0.764492
111  2026  10.598180
112  2026   0.764492
111  2027  10.674408
112  2027   0.764492
111  2028  10.748756
112  2028   0.764492
111  2029  10.822

In [159]:
conWork.url.database

'../DB/year.sqlite3'